In [7]:
import requests
import yaml, os
from pathlib import Path
from typing import List, Dict
from llama_index.llms import AzureOpenAI
from llama_index.llm_predictor import LLMPredictor
from llama_index import set_global_service_context
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.postprocessor import MetadataReplacementPostProcessor
from llama_index.response_synthesizers import get_response_synthesizer
from llama_index import ServiceContext, load_index_from_storage, StorageContext
from llama_index.indices.document_summary import DocumentSummaryIndexEmbeddingRetriever

In [2]:
with open('/Users/1zuu/Desktop/LLM RESEARCH/LLMPro/cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['AD_OPENAI_API_KEY'] = credentials['AD_OPENAI_API_KEY']

In [3]:
embedding_llm = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
llm=AzureOpenAI(
                deployment_name=credentials['AD_DEPLOYMENT_ID'],
                model=credentials['AD_ENGINE'],
                api_key=credentials['AD_OPENAI_API_KEY'],
                api_version=credentials['AD_OPENAI_API_VERSION'],
                azure_endpoint=credentials['AD_OPENAI_API_BASE']
                )
chat_llm = LLMPredictor(llm)

service_context = ServiceContext.from_defaults(
                                                embed_model=embedding_llm,
                                                llm_predictor=chat_llm
                                                )
set_global_service_context(service_context)

/Users/1zuu/miniforge3/envs/llamaindex/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
sentence_index = load_index_from_storage(StorageContext.from_defaults(persist_dir="./db/sentence_index"))
base_index = load_index_from_storage(StorageContext.from_defaults(persist_dir="./db/base_index"))

In [10]:
sentence_query_engine = sentence_index.as_query_engine(
                                                        similarity_top_k=2,
                                                        # the target key defaults to `window` to match the node_parser's default
                                                        node_postprocessors=[
                                                            MetadataReplacementPostProcessor(target_metadata_key="window")
                                                        ],
                                                    )

base_query_engine = base_index.as_query_engine(similarity_top_k=5)

In [11]:
window_response = sentence_query_engine.query("What are the concerns surrounding the AMOC?")
base_response = base_query_engine.query("What are the concerns surrounding the AMOC?")

In [13]:
str(window_response)

'There is low confidence in the quantification of AMOC changes in the 20th century due to low agreement in quantitative reconstructed and simulated trends. Direct observational records since the mid-2000s remain too short to determine the relative contributions of internal variability, natural forcing, and anthropogenic forcing to AMOC change. Over the 21st century, AMOC will very likely decline for all SSP scenarios but will not involve an abrupt collapse before 2100.'

In [16]:
str(base_response)

'There is low confidence in reconstructed and modelled AMOC changes for the 20th century, and the AMOC will decline over the 21st century with high confidence but low confidence for quantitative projections.'